# Pseudo Labeling Training

CSV 파일의 예측 결과를 pseudo label로 사용하여 학습하는 코드입니다.

## 목차
1. [Imports & Global Constants](#Imports-&-Global-Constants): 라이브러리 임포트 및 설정
2. [Utility Functions](#Utility-Functions): 유틸리티 함수 정의
3. [Dataset Classes](#Dataset-Classes): 데이터셋 클래스 정의
4. [Training Functions](#Training-Functions): 학습 함수 정의
5. [Load Checkpoint & Setup](#Load-Checkpoint-&-Setup): 체크포인트 로드 및 설정
6. [Training](#Training): 학습 실행


# Imports & Global Constants


In [1]:
# python native
import os
import json
import random
import datetime
from functools import partial
from collections import defaultdict
import gc

# external library
import cv2
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn.model_selection import GroupKFold
import albumentations as A

# torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torchvision import models
from torch.cuda.amp import autocast, GradScaler

# segmentation models
import segmentation_models_pytorch as smp

# wandb
import wandb


/opt/conda/lib/python3.10/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 2.0.8 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
wandb.login(key='b47be2b266addd7b64af746217d5a6ae7f428575')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /data/ephemeral/home/.netrc
wandb: Currently logged in as: 0129jonsu (cv_12) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
# 데이터 경로 설정
IMAGE_ROOT = "../data/train/DCM"
LABEL_ROOT = "../data/train/outputs_json"
PSEUDO_IMAGE_ROOT = "../data/test/DCM"  # Pseudo label 대상 이미지 경로

# CSV 파일 경로 (inference 결과가 담긴 CSV)
PSEUDO_CSV_PATH = "9754.csv"  # 여기에 pseudo label CSV 경로 입력

# 체크포인트 경로
CHECKPOINT_PATH = "checkpoints/efficientnetb3_DeepLabV3Plus_9733.pt"


In [4]:
CLASSES = [
    'finger-1', 'finger-2', 'finger-3', 'finger-4', 'finger-5',
    'finger-6', 'finger-7', 'finger-8', 'finger-9', 'finger-10',
    'finger-11', 'finger-12', 'finger-13', 'finger-14', 'finger-15',
    'finger-16', 'finger-17', 'finger-18', 'finger-19', 'Trapezium',
    'Trapezoid', 'Capitate', 'Hamate', 'Scaphoid', 'Lunate',
    'Triquetrum', 'Pisiform', 'Radius', 'Ulna',
]

CLASS2IND = {v: i for i, v in enumerate(CLASSES)}
IND2CLASS = {v: k for k, v in CLASS2IND.items()}


In [5]:
# 학습 설정
BATCH_SIZE = 2
ACCUMULATION_STEPS = 4
LR = 1e-5  # Pseudo labeling에서는 더 낮은 learning rate 권장
RANDOM_SEED = 21
NUM_EPOCHS = 50
VAL_EVERY = 1
PATIENCE = 10

# Pseudo label 데이터 사용 비율 (0.0 ~ 1.0)
PSEUDO_DATA_RATIO = 1.0

SAVED_DIR = "checkpoints"
if not os.path.exists(SAVED_DIR):
    os.makedirs(SAVED_DIR)

MODEL_NAME = "efficientnetb3_deeplabv3p_2048_pseudo.pt"

# Utility Functions


In [6]:
def decode_rle_to_mask(rle, height, width):
    """RLE로 인코딩된 결과를 mask map으로 복원합니다."""
    if pd.isna(rle) or rle == '':
        return np.zeros((height, width), dtype=np.uint8)
    
    s = rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(height * width, dtype=np.uint8)
    
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    
    return img.reshape(height, width)


In [7]:
def dice_coef(y_true, y_pred):
    y_true_f = y_true.flatten(2)
    y_pred_f = y_pred.flatten(2)
    intersection = torch.sum(y_true_f * y_pred_f, -1)
    
    eps = 0.0001
    return (2. * intersection + eps) / (torch.sum(y_true_f, -1) + torch.sum(y_pred_f, -1) + eps)


In [8]:
def save_model(model, file_name=MODEL_NAME):
    output_path = os.path.join(SAVED_DIR, file_name)
    torch.save(model, output_path)


In [9]:
def set_seed():
    torch.manual_seed(RANDOM_SEED)
    torch.cuda.manual_seed(RANDOM_SEED)
    torch.cuda.manual_seed_all(RANDOM_SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(RANDOM_SEED)
    random.seed(RANDOM_SEED)


# Dataset Classes


In [10]:
class XRayDataset(Dataset):
    """Original labeled dataset (JSON labels)"""
    def __init__(self, is_train=True, transforms=None):
        # 파일 목록 수집
        pngs = {
            os.path.relpath(os.path.join(root, fname), start=IMAGE_ROOT)
            for root, _dirs, files in os.walk(IMAGE_ROOT)
            for fname in files
            if os.path.splitext(fname)[1].lower() == ".png"
        }
        
        jsons = {
            os.path.relpath(os.path.join(root, fname), start=LABEL_ROOT)
            for root, _dirs, files in os.walk(LABEL_ROOT)
            for fname in files
            if os.path.splitext(fname)[1].lower() == ".json"
        }
        
        pngs = sorted(pngs)
        jsons = sorted(jsons)
        
        _filenames = np.array(pngs)
        _labelnames = np.array(jsons)
        
        # 핸드 타입(Left/Right) 자동 매핑 로직
        self.hand_side_map = {}
        files_by_folder = defaultdict(list)
        for fname in _filenames:
            folder = os.path.dirname(fname)
            files_by_folder[folder].append(fname)
            
        for folder, files in files_by_folder.items():
            files.sort()
            if len(files) > 0:
                self.hand_side_map[files[0]] = 'Right'
            if len(files) > 1:
                self.hand_side_map[files[1]] = 'Left'
        
        # split train-valid
        groups = [os.path.dirname(fname) for fname in _filenames]
        ys = [0 for fname in _filenames]
        
        gkf = GroupKFold(n_splits=5)
        
        filenames = []
        labelnames = []
        for i, (x, y) in enumerate(gkf.split(_filenames, ys, groups)):
            if is_train:
                if i == 0:  # 0번을 validation dataset으로 사용
                    continue
                filenames += list(_filenames[y])
                labelnames += list(_labelnames[y])
            else:
                filenames = list(_filenames[y])
                labelnames = list(_labelnames[y])
                break
        
        self.filenames = filenames
        self.labelnames = labelnames
        self.is_train = is_train
        self.transforms = transforms
        self.image_root = IMAGE_ROOT
        self.label_root = LABEL_ROOT
    
    def __len__(self):
        return len(self.filenames)
    
    def __getitem__(self, item):
        image_name = self.filenames[item]
        image_path = os.path.join(self.image_root, image_name)
        
        image = cv2.imread(image_path)
        image = image / 255.
        
        label_name = self.labelnames[item]
        label_path = os.path.join(self.label_root, label_name)
        
        label_shape = tuple(image.shape[:2]) + (len(CLASSES), )
        label = np.zeros(label_shape, dtype=np.uint8)
        
        with open(label_path, "r") as f:
            annotations = json.load(f)
        annotations = annotations["annotations"]
        
        for ann in annotations:
            c = ann["label"]
            class_ind = CLASS2IND[c]
            points = np.array(ann["points"])
            class_label = np.zeros(image.shape[:2], dtype=np.uint8)
            cv2.fillPoly(class_label, [points], 1)
            label[..., class_ind] = class_label
        
        # 오른손(Right)일 경우 Flip 적용
        hand_side = self.hand_side_map.get(image_name, 'Unknown')
        if hand_side == 'Right':
            image = cv2.flip(image, 1)
            label = cv2.flip(label, 1)
        
        if self.transforms is not None:
            inputs = {"image": image, "mask": label}
            result = self.transforms(**inputs)
            image = result["image"]
            label = result["mask"]

        image = image.transpose(2, 0, 1)
        label = label.transpose(2, 0, 1)
        
        image = torch.from_numpy(image).float()
        label = torch.from_numpy(label).float()
            
        return image, label


In [11]:
class PseudoLabelDataset(Dataset):
    """Pseudo label dataset using CSV predictions"""
    def __init__(self, csv_path, image_root, transforms=None, target_size=(2048, 2048)):
        self.image_root = image_root
        self.transforms = transforms
        self.target_size = target_size
        
        # CSV 파일 로드
        df = pd.read_csv(csv_path)
        
        # 이미지별로 그룹화
        self.image_names = df['image_name'].unique().tolist()
        
        # 각 이미지의 클래스별 RLE를 저장
        self.rle_dict = {}
        for img_name in self.image_names:
            img_df = df[df['image_name'] == img_name]
            self.rle_dict[img_name] = {}
            for _, row in img_df.iterrows():
                class_name = row['class']
                rle = row['rle']
                self.rle_dict[img_name][class_name] = rle
        
        # Pseudo data ratio 적용
        if PSEUDO_DATA_RATIO < 1.0:
            n_samples = int(len(self.image_names) * PSEUDO_DATA_RATIO)
            random.shuffle(self.image_names)
            self.image_names = self.image_names[:n_samples]
        
        # 핸드 타입 매핑 (폴더별로)
        self.hand_side_map = self._build_hand_side_map()
        
        print(f"[PseudoLabelDataset] Loaded {len(self.image_names)} images from CSV")
    
    def _build_hand_side_map(self):
        """폴더 내 파일 순서로 Left/Right 결정"""
        hand_side_map = {}
        
        # 이미지 경로에서 폴더별로 그룹화
        files_by_folder = defaultdict(list)
        
        # image_root 내의 모든 png 파일 탐색
        for root, _dirs, files in os.walk(self.image_root):
            for fname in files:
                if fname.lower().endswith('.png'):
                    rel_path = os.path.relpath(os.path.join(root, fname), start=self.image_root)
                    folder = os.path.dirname(rel_path)
                    files_by_folder[folder].append(rel_path)
        
        for folder, files in files_by_folder.items():
            files.sort()
            if len(files) > 0:
                # 파일명만 추출해서 매핑
                basename = os.path.basename(files[0])
                hand_side_map[basename] = 'Right'
            if len(files) > 1:
                basename = os.path.basename(files[1])
                hand_side_map[basename] = 'Left'
        
        return hand_side_map
    
    def _find_image_path(self, image_name):
        """이미지 이름으로 전체 경로 찾기"""
        for root, _dirs, files in os.walk(self.image_root):
            for fname in files:
                if fname == image_name:
                    return os.path.join(root, fname)
        return None
    
    def __len__(self):
        return len(self.image_names)
    
    def __getitem__(self, item):
        image_name = self.image_names[item]
        image_path = self._find_image_path(image_name)
        
        if image_path is None:
            raise FileNotFoundError(f"Image not found: {image_name}")
        
        image = cv2.imread(image_path)
        orig_h, orig_w = image.shape[:2]
        image = image / 255.
        
        # CSV에서 pseudo label 생성
        label = np.zeros((self.target_size[0], self.target_size[1], len(CLASSES)), dtype=np.uint8)
        
        rle_data = self.rle_dict.get(image_name, {})
        for class_name, rle in rle_data.items():
            if class_name in CLASS2IND:
                class_idx = CLASS2IND[class_name]
                mask = decode_rle_to_mask(rle, self.target_size[0], self.target_size[1])
                label[..., class_idx] = mask
        
        # 오른손(Right)일 경우 Flip 적용 (학습 때와 동일하게)
        hand_side = self.hand_side_map.get(image_name, 'Unknown')
        if hand_side == 'Right':
            image = cv2.flip(image, 1)
            label = cv2.flip(label, 1)
        
        if self.transforms is not None:
            inputs = {"image": image, "mask": label}
            result = self.transforms(**inputs)
            image = result["image"]
            label = result["mask"]

        image = image.transpose(2, 0, 1)
        label = label.transpose(2, 0, 1)
        
        image = torch.from_numpy(image).float()
        label = torch.from_numpy(label).float()
            
        return image, label


# Training Functions


In [12]:
def validation(epoch, model, data_loader, criterion, thr=0.5):
    print(f'Start validation #{epoch:2d}')
    model.eval()
    model = model.cuda()

    dices = []
    total_loss = 0
    cnt = 0
    
    with torch.no_grad():
        for step, (images, masks) in tqdm(enumerate(data_loader), total=len(data_loader)):
            images, masks = images.cuda(), masks.cuda()         
            
            outputs = model(images)
            
            output_h, output_w = outputs.size(-2), outputs.size(-1)
            mask_h, mask_w = masks.size(-2), masks.size(-1)
            
            if output_h != mask_h or output_w != mask_w:
                outputs = F.interpolate(outputs, size=(mask_h, mask_w), mode="bilinear")
            
            loss = criterion(outputs, masks)
            total_loss += loss.item()
            cnt += 1
            
            outputs = torch.sigmoid(outputs)
            outputs = (outputs > thr).detach()
            masks = masks.detach()
            
            dice = dice_coef(outputs, masks)
            dices.append(dice.cpu())
                
    dices = torch.cat(dices, 0)
    dices_per_class = torch.mean(dices, 0)
    dice_str = [
        f"{c:<12}: {d.item():.4f}"
        for c, d in zip(CLASSES, dices_per_class)
    ]
    dice_str = "\n".join(dice_str)
    print(dice_str)
    
    avg_dice = torch.mean(dices_per_class).item()
    avg_loss = total_loss / cnt

    wandb_logs = {
        "Val/Mean_Dice": avg_dice,
        "Val/Loss": avg_loss,
        "Epoch": epoch
    }

    for class_name, score in zip(CLASSES, dices_per_class):
        wandb_logs[f"Val/Dice_{class_name}"] = score.item()
    
    wandb.log(wandb_logs)
    
    return avg_dice


In [13]:
def train(model, data_loader, val_loader, criterion, optimizer, patience=PATIENCE):
    print(f'Start training with Mixed Precision (FP16) and Pseudo Labels..')
    
    best_dice = 0.
    model = model.cuda()
    patience_check = 0
    
    scaler = GradScaler()
    optimizer.zero_grad()
    
    for epoch in range(NUM_EPOCHS):
        gc.collect()
        torch.cuda.empty_cache()
        
        model.train()
        epoch_loss = 0.0

        loop = tqdm(enumerate(data_loader), total=len(data_loader), desc=f"Epoch {epoch+1}/{NUM_EPOCHS}")
        
        for step, (images, masks) in loop:            
            images, masks = images.cuda(), masks.cuda()
            
            with autocast():
                outputs = model(images)
                loss = criterion(outputs, masks)
                loss = loss / ACCUMULATION_STEPS
            
            scaler.scale(loss).backward()

            if (step + 1) % ACCUMULATION_STEPS == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
            
            current_loss = loss.item() * ACCUMULATION_STEPS
            
            wandb.log({
                "Train/Step_Total_Loss": current_loss,
                "Train/Learning_Rate": optimizer.param_groups[0]['lr']
            })
            
            epoch_loss += current_loss

            loop.set_postfix({
                "Loss": f"{loss.item():.4f}",
                "LR": f"{optimizer.param_groups[0]['lr']:.6f}"
            })
            
            if (step + 1) % 25 == 0:
                print(
                    f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} | '
                    f'Epoch [{epoch+1}/{NUM_EPOCHS}], '
                    f'Step [{step+1}/{len(data_loader)}], '
                    f'Loss: {round(current_loss,4)}'
                )
                
        avg_loss = epoch_loss / len(data_loader)
        wandb.log({"Train/Epoch_Loss": avg_loss, "Epoch": epoch + 1})
             
        if (epoch + 1) % VAL_EVERY == 0:
            dice = validation(epoch + 1, model, val_loader, criterion)
            
            if best_dice < dice:
                print(f"Best performance at epoch: {epoch + 1}, {best_dice:.4f} -> {dice:.4f}")
                print(f"Save model in {SAVED_DIR}")
                best_dice = dice
                save_model(model)
                patience_check = 0
            else:
                patience_check += 1
                print(f"Early Stopping Counter: {patience_check}/{patience}")
                
                if patience_check >= patience:
                    print(f"Early stopping triggered at epoch {epoch + 1}!")
                    break


# Load Checkpoint & Setup


In [14]:
# 시드 설정
set_seed()

# 체크포인트 로드
print(f"Loading checkpoint from: {CHECKPOINT_PATH}")
if os.path.exists(CHECKPOINT_PATH):
    model = torch.load(CHECKPOINT_PATH)
    print("Checkpoint loaded successfully!")
else:
    print(f"Checkpoint not found at {CHECKPOINT_PATH}. Creating new model...")
    encoder_name = "efficientnet-b3"
    model = smp.UnetPlusPlus(
        encoder_name=encoder_name,    
        encoder_weights="imagenet",
        in_channels=3,  
        classes=len(CLASSES)      
    )


Loading checkpoint from: checkpoints/efficientnetb3_DeepLabV3Plus_9733.pt
Checkpoint loaded successfully!


In [15]:
# 데이터 변환 정의
train_tf = A.Compose([
    A.Resize(2048, 2048),
    A.ShiftScaleRotate(
        shift_limit=0.1, 
        scale_limit=0.2, 
        rotate_limit=30, 
        border_mode=cv2.BORDER_CONSTANT, 
        value=0, mask_value=0,
        p=0.5
    ),
    
    A.RandomBrightnessContrast(
        brightness_limit=0.2, 
        contrast_limit=0.2, 
        p=0.5
    ),
    
    A.CoarseDropout(
        max_holes=10,
        max_height=128,
        max_width=128,
        min_holes=2,
        min_height=32,
        min_width=32,
        fill_value=0,
        mask_fill_value=None,
        p=0.3
    ),
])

valid_tf = A.Compose([
    A.Resize(2048, 2048),
])


In [16]:
# Original labeled dataset 로드
print("Loading original labeled dataset...")
train_dataset = XRayDataset(is_train=True, transforms=train_tf)
valid_dataset = XRayDataset(is_train=False, transforms=valid_tf)
print(f"Original train samples: {len(train_dataset)}")
print(f"Original valid samples: {len(valid_dataset)}")


Loading original labeled dataset...
Original train samples: 640
Original valid samples: 160


In [17]:
# Pseudo label dataset 로드
print(f"\nLoading pseudo label dataset from: {PSEUDO_CSV_PATH}")
if os.path.exists(PSEUDO_CSV_PATH):
    pseudo_dataset = PseudoLabelDataset(
        csv_path=PSEUDO_CSV_PATH,
        image_root=PSEUDO_IMAGE_ROOT,
        transforms=train_tf,
        target_size=(2048, 2048)
    )
    print(f"Pseudo label samples: {len(pseudo_dataset)}")
    
    # Original + Pseudo 데이터 합치기
    combined_dataset = ConcatDataset([pseudo_dataset])
    print(f"Combined train samples: {len(combined_dataset)}")
else:
    print(f"Warning: Pseudo CSV not found at {PSEUDO_CSV_PATH}")
    print("Training with original data only.")
    combined_dataset = train_dataset



Loading pseudo label dataset from: 9754.csv
[PseudoLabelDataset] Loaded 288 images from CSV
Pseudo label samples: 288
Combined train samples: 288


In [18]:
# DataLoader 생성
train_loader = DataLoader(
    dataset=combined_dataset, 
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    drop_last=True,
)

valid_loader = DataLoader(
    dataset=valid_dataset, 
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4,
    drop_last=False
)

print(f"Train loader batches: {len(train_loader)}")
print(f"Valid loader batches: {len(valid_loader)}")


Train loader batches: 144
Valid loader batches: 80


In [19]:
# Loss 및 Optimizer 설정
dice_loss_fn = smp.losses.DiceLoss(mode='multilabel', from_logits=True)
bce_loss_fn = nn.BCEWithLogitsLoss()

def criterion(y_pred, y_true):
    loss_bce = bce_loss_fn(y_pred, y_true)
    loss_dice = dice_loss_fn(y_pred, y_true)
    return 0.5 * loss_bce + 0.5 * loss_dice

optimizer = optim.Adam(params=model.parameters(), lr=LR, weight_decay=1e-6)


In [20]:
# WandB 초기화
wandb.init(
    project="deeplab_v3_plus", 
    name=f"{MODEL_NAME}",
    config={
        "epochs": NUM_EPOCHS,
        "lr": LR,
        "batch_size": BATCH_SIZE,
        "pseudo_data_ratio": PSEUDO_DATA_RATIO,
        "checkpoint": CHECKPOINT_PATH
    }
)


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


# Training


In [21]:
# 학습 시작
print("=" * 50)
print("Starting Pseudo Label Training...")
print("=" * 50)

train(model, train_loader, valid_loader, criterion, optimizer)


Starting Pseudo Label Training...
Start training with Mixed Precision (FP16) and Pseudo Labels..


Epoch 1/50:   0%|          | 0/144 [00:00<?, ?it/s]

2026-01-06 01:03:58 | Epoch [1/50], Step [25/144], Loss: 0.0086
2026-01-06 01:04:43 | Epoch [1/50], Step [50/144], Loss: 0.0085
2026-01-06 01:05:29 | Epoch [1/50], Step [75/144], Loss: 0.0103
2026-01-06 01:06:12 | Epoch [1/50], Step [100/144], Loss: 0.0076
2026-01-06 01:06:57 | Epoch [1/50], Step [125/144], Loss: 0.0147
Start validation # 1


  0%|          | 0/80 [00:00<?, ?it/s]

finger-1    : 0.9698
finger-2    : 0.9839
finger-3    : 0.9880
finger-4    : 0.9771
finger-5    : 0.9775
finger-6    : 0.9877
finger-7    : 0.9858
finger-8    : 0.9792
finger-9    : 0.9809
finger-10   : 0.9892
finger-11   : 0.9796
finger-12   : 0.9783
finger-13   : 0.9805
finger-14   : 0.9875
finger-15   : 0.9824
finger-16   : 0.9738
finger-17   : 0.9718
finger-18   : 0.9855
finger-19   : 0.9864
Trapezium   : 0.9513
Trapezoid   : 0.9233
Capitate    : 0.9696
Hamate      : 0.9567
Scaphoid    : 0.9728
Lunate      : 0.9660
Triquetrum  : 0.9560
Pisiform    : 0.9149
Radius      : 0.9913
Ulna        : 0.9896
Best performance at epoch: 1, 0.0000 -> 0.9737
Save model in checkpoints


...

finger-1    : 0.9698
finger-2    : 0.9840
finger-3    : 0.9881
finger-4    : 0.9770
finger-5    : 0.9778
finger-6    : 0.9877
finger-7    : 0.9858
finger-8    : 0.9791
finger-9    : 0.9812
finger-10   : 0.9891
finger-11   : 0.9797
finger-12   : 0.9784
finger-13   : 0.9804
finger-14   : 0.9874
finger-15   : 0.9825
finger-16   : 0.9739
finger-17   : 0.9719
finger-18   : 0.9855
finger-19   : 0.9865
Trapezium   : 0.9530
Trapezoid   : 0.9269
Capitate    : 0.9701
Hamate      : 0.9582
Scaphoid    : 0.9742
Lunate      : 0.9667
Triquetrum  : 0.9568
Pisiform    : 0.9184
Radius      : 0.9913
Ulna        : 0.9895
Early Stopping Counter: 9/10


Epoch 45/50:   0%|          | 0/144 [00:00<?, ?it/s]

2026-01-06 05:40:54 | Epoch [45/50], Step [25/144], Loss: 0.0057
2026-01-06 05:41:37 | Epoch [45/50], Step [50/144], Loss: 0.0047
2026-01-06 05:42:23 | Epoch [45/50], Step [75/144], Loss: 0.0084
2026-01-06 05:43:10 | Epoch [45/50], Step [100/144], Loss: 0.0053
2026-01-06 05:43:56 | Epoch [45/50], Step [125/144], Loss: 0.0083
Start validation #45


  0%|          | 0/80 [00:00<?, ?it/s]

finger-1    : 0.9697
finger-2    : 0.9840
finger-3    : 0.9881
finger-4    : 0.9771
finger-5    : 0.9778
finger-6    : 0.9877
finger-7    : 0.9858
finger-8    : 0.9792
finger-9    : 0.9811
finger-10   : 0.9891
finger-11   : 0.9795
finger-12   : 0.9784
finger-13   : 0.9804
finger-14   : 0.9873
finger-15   : 0.9824
finger-16   : 0.9740
finger-17   : 0.9719
finger-18   : 0.9855
finger-19   : 0.9864
Trapezium   : 0.9532
Trapezoid   : 0.9263
Capitate    : 0.9700
Hamate      : 0.9583
Scaphoid    : 0.9740
Lunate      : 0.9666
Triquetrum  : 0.9570
Pisiform    : 0.9196
Radius      : 0.9913
Ulna        : 0.9895
Early Stopping Counter: 10/10
Early stopping triggered at epoch 45!


In [22]:
# WandB 종료
wandb.finish()
print("\nTraining completed!")


Epoch,▁▁▁▁▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇█████
Train/Epoch_Loss,█▇▆▆▅▄▄▄▄▄▄▄▄▄▃▃▃▃▄▃▄▃▂▃▂▃▂▁▂▃▁▁▁▂▂▁▁▁▂▂
Train/Learning_Rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train/Step_Total_Loss,▅▅█▇▄▅▅▁▁▄▅▂▃▃▇▃▂▂▅▄▆▁▃▁▅▂▄▃▂▅▄▄▂▂▃▂▃▆▃▂
Val/Dice_Capitate,▁▁▂▂▁▃▂▄▃▃▃▄▄▃▄▄▅▄▄▅▅▅▅▆▆▆▅▅▆▆▆▇▇▇▆▆███▇
Val/Dice_Hamate,▁▂▄▃▃▅▄▆▅▅▅▆▆▅▇▆▇▆▆▇▆▇▇▇▆▇▇▇▇▆▆▇▇█▆▇▇▆▇█
Val/Dice_Lunate,▂▂▂▁▂▃▂▅▃▄▄▃▃▅▄▄▆▆▆▅▅▅▆▇▆▇▆▆▇▆▆▇▇█▇▆▇▇█▆
Val/Dice_Pisiform,▁▂▂▄▅▅▃▆▅▅▆▇▅▅▄▆▆▆▇▆▆▇▇▆▇▇▅▆▆▆▆█▅▇▆▆▆▆▆▇
Val/Dice_Radius,▁▁▄▃▃▂▄▄▄▁▂▁▃▆▆▄▅▅▂▄▄▄▃▃▄▂▃█▅▇▆▅▄▄▇▆▆▄▃▂
Val/Dice_Scaphoid,▁▂▃▅▄▅▅▆▄▃▅▇▆▇▆▆▇▆█▇▇█▇▆▇▆▇▆█▆▇▇▆█▇▇▆██▇
+25,...



Training completed!
